In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
# Testing load graph from shapefile
deep_reload(src)

node_attributes={}

link_attributes={
    'length': lambda e: e['LENGTH']*1e3,
    'speed': lambda e: e['SPEEDLIM']/3.6,
    'time': lambda e: (e['LENGTH']*1e3)/(e['SPEEDLIM']/3.6),
}

roadmap = src.graph.graph_from_shapefile(
    'RoadMap/roadmap.shp',
    node_attributes = node_attributes,
    link_attributes = link_attributes,
)

In [3]:
# Testing save graph to json
deep_reload(src)

filename = 'RoadMap/roadmap.json'

src.graph.graph_to_json(roadmap, filename)

In [4]:
# Testing load graph from json
deep_reload(src)

filename = 'RoadMap/roadmap.json'

roadmap = src.graph.graph_from_json(filename)

In [28]:
# Testing loading data from csv
deep_reload(src)

filename=[
    'CEC/ACL2_Weighted_Sample.csv',
    'CEC/DCL2_Weighted_Sample.csv',
    'CEC/Depots.csv',
]

node_attributes={
    'x': lambda r: r['Longitude'],
    'y': lambda r: r['Latitude'],
    'ev_network_clean': lambda r: r['EV Network Clean'],
}

graph = src.graph.graph_from_csv(filename, node_attributes)

In [8]:
# Testing graph-to-graph assignment
deep_reload(src)

x, y = np.array([[n['x'], n['y']] for n in graph._node.values()]).T

node_assignment = src.adjacency.closest_nodes_from_coordinates(roadmap, x, y)

In [9]:
node_assignment[:4]

[{'id': 27221,
  'query': array([-121.552662,   37.007051]),
  'result': array([-121.551502,   37.004138])},
 {'id': 30687,
  'query': array([-118.4614221 ,   34.03997437]),
  'result': array([-118.463337,   34.040543])},
 {'id': 18203,
  'query': array([-122.0595171 ,   37.97246804]),
  'result': array([-122.060028,   37.973517])},
 {'id': 50225,
  'query': array([-121.370537,   38.380569]),
  'result': array([-121.365632,   38.380197])}]

In [14]:
# Testing routing along map single O/D
deep_reload(src)

sources = node_assignment[0]['id']
targets = [node_assignment[1000]['id']]

weights = {
    'length': 300e3,
    'time': 3600*8
}
    

res = src.adjacency.single_source_dijkstra(
    roadmap,
    sources,
    targets,
    weights,
)

res

[{'source': 27221,
  'target': 30286,
  'length': 185170.0,
  'time': 7285.1302697302735}]

In [27]:
# Testing routing along map multiple O/D
deep_reload(src)

rng = np.random.default_rng(1030)

sources = node_assignment[0]['id']
targets = [node_assignment[idx]['id'] for idx in rng.integers(0, len(node_assignment), 5)]

weights = {
    'length': 300e3,
    'time': 3600*8,
}
    

result = src.adjacency.single_source_dijkstra(
    roadmap,
    sources,
    targets,
    weights,
)

result

[{'source': 27221,
  'target': 9756,
  'length': 99780.0,
  'time': 4498.959740259741},
 {'source': 27221,
  'target': 24665,
  'length': 214290.0,
  'time': 7720.8333166833145},
 {'source': 27221,
  'target': 49910,
  'length': 238720.0,
  'time': 8798.8299825175}]

In [142]:
# Testing routing along map multiple O and multiple D
deep_reload(src)

rng = np.random.default_rng(1030)

sources = [node_assignment[idx]['id'] for idx in rng.integers(0, len(node_assignment), 5)]
targets = [node_assignment[idx]['id'] for idx in rng.integers(0, len(node_assignment), 5)]

weights = {
    'length': 300e3,
    'time': 3600*8,
}
    

result = src.adjacency.multiple_source_dijkstra(
    roadmap,
    sources,
    targets,
    weights,
    pb_kwargs = {'color': '\33[34m'},
)

 Progress [------------>] (5/5) 100.00%, 9.91 it/s, 0.61 s elapsed, 0.00 s remaining



In [143]:
# Testing adding adjacency to graph
deep_reload(src)

rng = np.random.default_rng(1030)

sources = rng.choice(list(graph.nodes), 15)

subgraph = src.graph.subgraph(graph, sources)

for n in subgraph._node.values():
    n['status']=0

weights = {
    'length': 300e3,
    'time': 3600*8,
}

subgraph = src.adjacency.adjacency(roadmap, subgraph, weights)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/aaron/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_133517/1590644538.py", line 18, in <module>
    subgraph = src.adjacency.adjacency(roadmap, subgraph, weights)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aaron/Desktop/electric_vrp/src/adjacency.py", line 175, in adjacency
    results = multiple_source_dijkstra(atlas, sources, targets, weights, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aaron/Desktop/electric_vrp/src/adjacency.py", line 106, in multiple_source_dijkstra
    for source in ProgressBar(sources, **kwargs['pb_kwargs']):
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ProgressBar.__init__() got an unexpected keyword argument 'verbose'

During handling of the above exception, a